<a href="https://colab.research.google.com/github/kojikokojiko/portfoilio/blob/master/LSTM_future_sale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/My Drive/Colab Notebooks/Kaggle/Predict Future Sales

/content/drive/My Drive/Colab Notebooks/Kaggle/Predict Future Sales


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, ThresholdedReLU, MaxPooling2D, Embedding, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras import backend as K
import keras


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
items = pd.read_csv('./input/items.csv')
shops = pd.read_csv('./input/shops.csv')
cats = pd.read_csv('./input/item_categories.csv')
sales= pd.read_csv('./input/sales_train.csv', parse_dates=['date'], infer_datetime_format=True, dayfirst=True)
val  = pd.read_csv('./input/test.csv')

##月の売り上げ個数の時系列データを作る

In [0]:
dataset_sales=sales.pivot_table(index=['shop_id','item_id'],values=['item_cnt_day'],columns=['date_block_num'],fill_value=0,aggfunc='sum')

In [0]:
dataset_sales.reset_index(inplace = True)

In [6]:
df_sales=pd.merge(val,dataset_sales,on=['shop_id','item_id'],how='left')

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [0]:
df_sales.fillna(0,inplace=True)

In [0]:
df_sales.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)

##priceの月平均の時系列データをつくる

In [0]:
# スケールを0から1の間に収める
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
sales["item_price"] = scaler.fit_transform(sales["item_price"].values.reshape(-1,1))

In [0]:
dataset_price=sales.pivot_table(index=['shop_id','item_id'],columns=['date_block_num'],values=['item_price'],aggfunc='mean').fillna(0).reset_index()

In [11]:
df_price=pd.merge(val,dataset_price,on=['shop_id','item_id'],how='left')

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [0]:
df_price.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)

In [0]:
df_price.fillna(0,inplace=True)

## trainデータ作成

In [0]:
y_train=df_sales.iloc[:,-1]
x_sales_train=df_sales.iloc[:,:-1]
x_price_train=df_price.iloc[:,:-1]

In [15]:
x_price_train.values.shape

(214200, 33)

In [0]:
x_sales_train=x_sales_train.values.reshape((x_sales_train.shape[0],x_sales_train.shape[1],1))
x_price_train=x_price_train.values.reshape((x_price_train.shape[0],x_price_train.shape[1],1))


In [17]:
x_sales_train.shape

(214200, 33, 1)

In [18]:
x_price_train.shape

(214200, 33, 1)

In [19]:
y_train

0         0.0
1         0.0
2         1.0
3         0.0
4         0.0
         ... 
214195    1.0
214196    0.0
214197    0.0
214198    0.0
214199    0.0
Name: (item_cnt_day, 33), Length: 214200, dtype: float64

In [44]:
y_train.shape

(214200, 1)

In [0]:
y_train=y_train.values.reshape((214200,1))

In [0]:
X_train=np.append(x_sales_train,x_price_train,axis=2)

In [22]:
X_train.shape

(214200, 33, 2)

##testデータ作成

In [0]:
x_sales_test=df_sales.iloc[:,1:]
x_price_test=df_price.iloc[:,1:]


In [0]:
x_sales_test=np.expand_dims(x_sales_test.values,axis=2)

In [0]:
x_price_test=np.expand_dims(x_price_test.values,axis=2)

In [31]:
x_price_test.shape

(214200, 33, 1)

In [0]:
X_test=np.append(x_sales_test,x_price_test,axis=2)

##model作成

In [62]:
model= Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(optimizer="adam", loss='mse', metrics=["mse"])
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 33, 64)            17152     
_________________________________________________________________
dropout_10 (Dropout)         (None, 33, 64)            0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 33        
Total params: 29,601
Trainable params: 29,601
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
hist=model.fit(X_train,y_train,
               batch_size=4096,
               verbose=2,
               epochs=8


               )

Epoch 1/8
 - 90s - loss: 30.5197 - mse: 30.5197
Epoch 2/8
 - 88s - loss: 30.3027 - mse: 30.3027
Epoch 3/8
 - 88s - loss: 30.1328 - mse: 30.1328
Epoch 4/8
 - 89s - loss: 30.0813 - mse: 30.0813
Epoch 5/8
 - 92s - loss: 30.0556 - mse: 30.0556
Epoch 6/8
 - 88s - loss: 29.9741 - mse: 29.9741
Epoch 7/8
 - 88s - loss: 29.9296 - mse: 29.9296
Epoch 8/8
 - 88s - loss: 29.9477 - mse: 29.9477


In [0]:
y_pred=model.predict(X_test).clip(0,20).round()

In [0]:
submission = pd.DataFrame(y_pred,columns=['item_cnt_month'])
submission.to_csv('time_lstm2_round.csv',index_label='ID')